In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
#import train as train

import loss as loss
import unet as unet
import random
import paths as paths
import numpy as np
import glob

# The meaning of lifes
random.seed(42)  
np.random.seed(42)

In [ ]:
DATA_MEAN = 191.46748269704375
DATA_STD = 369.2190429494859
BATCH_SIZE = 1
OUTPUT_CHANNELS = 3
DATA_PATH = "/home/matthew/masters_code/dataset_prostate_cleaned/"

In [ ]:
#WEIGHTS = "./prostate_tversky/tvsky_epoch_" + "42" + ".hdf5"
#WEIGHTS = "./prostate_bce/bce_epoch_" + "64" + ".hdf5"
#WEIGHTS = "./prostate_dsc/dsc_epoch_" + "59" + ".hdf5"
WEIGHTS = "./prostate_wdsc/wdsc_after_bce_epoch_" + "249" + ".hdf5"
#WEIGHTS = "./prostate_bce_wdsc/bce_wdsc_epoch_78.hdf5"

INITIAL_LR = 1e-6
OPTIMIZER = tf.keras.optimizers.Adam(lr = INITIAL_LR)
LOSS = loss.weighted_dsc_loss
METRICS = [loss.dice_metric,  loss.dsc,
        tf.keras.metrics.Precision(), 
        tf.keras.metrics.Recall()]

model = unet.model(output_channels=OUTPUT_CHANNELS)
model.compile(OPTIMIZER, LOSS, METRICS)
model.load_weights(WEIGHTS)

In [ ]:
patient_paths = paths.get_patient_paths(DATA_PATH)
patient_paths.sort()

img_paths = [glob.glob(path + "/img/*") for path in patient_paths]
mask_paths = [glob.glob(path + "/mask/*") for path in patient_paths]

valid = int(len(img_paths) * 0.15 // 1)
test = int(len(img_paths) * 0.1 // 1)
train = int(len(img_paths) - valid - test)

train_inputs = paths.flatten_list(img_paths[0:train])
train_truths = paths.flatten_list(mask_paths[0:train])

train_inputs.sort()
train_truths.sort()

valid_inputs = paths.flatten_list(img_paths[train:train+valid])
valid_truths = paths.flatten_list(mask_paths[train:train+valid])

valid_inputs.sort()
valid_truths.sort()

test_inputs = paths.flatten_list(img_paths[train+valid:])
test_truths = paths.flatten_list(mask_paths[train+valid:])

test_inputs.sort()
test_truths.sort()

In [ ]:
# all_truths = paths.flatten_list(mask_paths)
# all_truths = np.array([np.load(array) for array in all_truths])
# print(all_truths.shape)
# img_size = 512*512
# images = all_truths.shape[0]

In [ ]:
# def perc_images(outputs, organ_index):
#     images = outputs.shape[0]
#     occurs = 0
#     for x in outputs[...,organ_index]:
#         if x.any() == 1:
#             occurs += 1
#     return (occurs / images) * 100

# def mask_image_perc(outputs, organ_index):
#     pixels = 0
#     occurs = 0
#     for x in outputs[...,organ_index]:
#         if x.any() == 1:
#             occurs += 1
#             pixels += np.sum(x==1)
#     return (pixels / (occurs * 512 * 512))*100

# def mask_output_perc(outputs, organ_index):
#     return mask_image_perc(outputs, organ_index) / outputs.shape[-1]

# def pixels_total_perc(outputs, organ_index):
#     pixels = 0
#     for x in outputs[...,organ_index]:
#         pixels += np.sum(x==1)
#     return (pixels / np.prod(outputs.shape))*100
        

In [ ]:
# print(perc_images(all_truths, 0))
# print(perc_images(all_truths, 1))
# print(perc_images(all_truths, 2))

In [ ]:
# print(mask_image_perc(all_truths, 0))
# print(mask_image_perc(all_truths, 1))
# print(mask_image_perc(all_truths, 2))

In [ ]:
# print(mask_output_perc(all_truths, 0))
# print(mask_output_perc(all_truths, 1))
# print(mask_output_perc(all_truths, 2))

In [ ]:
# print(pixels_total_perc(all_truths, 0))
# print(pixels_total_perc(all_truths, 1))
# print(pixels_total_perc(all_truths, 2))

In [ ]:
# total_pixels = np.prod(all_truths.shape)


# organ = 0
# for img in all_truths[...,2]:
#     if img.any() > 0.5:
#         organ += 1
        
# print(organ/all_truths.shape[0])


# num_pos = np.sum(all_truths > 0 )
# print((num_pos / total_pixels)*100)

In [ ]:
test_inputs = np.array([np.load(array) for array in test_inputs])
test_truths = np.array([np.load(array) for array in test_truths])

In [ ]:
test_inputs = (test_inputs - DATA_MEAN) / DATA_STD

In [ ]:
test_inputs.shape, test_truths.shape

In [ ]:
test_inputs.min(), test_inputs.max(), test_inputs.std(), test_inputs.mean()

In [ ]:
model.evaluate(x=test_inputs, y=test_truths, batch_size=1)

In [ ]:
spacing_mm = np.array([1.3671875, 1.3671875, 2])

rectum_mean = 2.862
rectum_std = 2.066

bladder_mean = 0.99
bladder_std = np.std(np.array([.23, .2, .1, .76, .21]))

# patient_tol_mm = 1
rectum_tol_mm = rectum_mean + 2 * rectum_std However, unlike
bladder_tol_mm = bladder_mean + 2 * bladder_std
rectum_tol_mm, bladder_tol_mm

In [ ]:
from matplotlib import pyplot as plt
import cv2
from skimage import exposure

# number=5
# random_indexes = np.random.randint(0, test_inputs.shape[0], number)
# print(random_indexes)


organ_name = "patient"
organ_num = 0
random_indexes = [130, 55, 14, 8, 83]
#random_indexes = [0, 90, 26, 43, 106]
number = len(random_indexes)
tol = 1.3671875
x1, x2, y1, y2 = 100, 100, 100, 100

# organ_name = "bladder"
# organ_num = 1
# random_indexes = [25, 16, 96, 24, 93]
# #random_indexes = np.arange(1,100)
# number = len(random_indexes)
# tol = bladder_tol_mm
# x1, x2, y1, y2 = 190, 190, 150, 230

# random_indexes = [42, 128, 70, 27, 131]
# number = len(random_indexes)
# organ_name = "rectum"
# organ_num = 2
# tol = rectum_tol_mm
# x1, x2, y1, y2 = 220, 220, 220, 220




fig, axs = plt.subplots(nrows=number, ncols=4, sharex=True, sharey=True,squeeze=True, figsize=(12,12))
for j, index in enumerate(random_indexes):
    ax = axs[j]
    
    for a in ax:
        a.set_xticklabels([])
        a.set_yticklabels([])
        a.set_xticks([])
        a.set_yticks([])
        
    img = test_inputs[index:index+1,...].copy()
    truth = test_truths[index:index+1,...,organ_num:organ_num+1]
    pred = model.predict(img)
    pred = np.round(pred[...,organ_num:organ_num+1])
    diff = truth - pred
#     img[img<0] = 0
#     img[img>0.4] = 1
    img[img>2.8] = img.max()
    img[img<2.0] = img.min()
    eq = []
    for im in img:
        img_eq = exposure.equalize_hist(im)
        #img_eq = exposure.equalize_adapthist(im, clip_limit=0.03)
        #p2, p98 = np.percentile(im, (7, 99))
        #img_eq = exposure.rescale_intensity(im, in_range=(p2, p98))
        eq.append(img_eq)
    img = np.array(eq)

    
    if np.max(truth) >= 1 and np.max(pred) >= 1:
        stats = google_metrics(truth, pred, spacing_mm, tol)
        print(stats)   
        stats = "DSC: {0:.3f}\nsDSC: {1:.3f}\nMSD: {2:.3f}".format(stats[0][0], stats[1][0], stats[2][0])
        #stats = "DSC: {0:.3f}\nMSD: {1:.3f}".format(stats[0][0], stats[2][0])
        ax[3].text(1.05, 0.4, stats, size=18, ha="left", transform=ax[3].transAxes)

    
    ax[0].imshow(img[0,...,0][y1:-y2, x1:-x2], cmap='gray')
    ax[0].contour(truth[0,...,0][y1:-y2, x1:-x2], colors='yellow', alpha=.25)
    ax[0].contour(pred[0,...,0][y1:-y2, x1:-x2],colors='red', alpha=.25)
    ax[1].imshow(truth[0,...,0][y1:-y2, x1:-x2])
    ax[2].imshow(pred[0,...,0][y1:-y2, x1:-x2])
    ax[3].imshow(diff[0,...,0][y1:-y2, x1:-x2])

plt.tight_layout()

axs[0][0].set_title('Input crop', fontdict={'fontsize': 23, 'fontweight': 'medium'})
axs[0][1].set_title('Truth (Y)', fontdict={'fontsize': 23, 'fontweight': 'medium'})
axs[0][2].set_title('Prediction (R)', fontdict={'fontsize': 23, 'fontweight': 'medium'})
axs[0][3].set_title('Difference', fontdict={'fontsize': 23, 'fontweight': 'medium'})

axs[0][0].text(-0.2,0.5, "A", size=23, ha="center", transform=axs[0][0].transAxes)
axs[1][0].text(-0.2,0.5, "B", size=23, ha="center", transform=axs[1][0].transAxes)
axs[2][0].text(-0.2,0.5, "C", size=23, ha="center", transform=axs[2][0].transAxes)
axs[3][0].text(-0.2,0.5, "D", size=23, ha="center", transform=axs[3][0].transAxes)
axs[4][0].text(-0.2,0.5, "E", size=23, ha="center", transform=axs[4][0].transAxes)

#plt.savefig("prostate_BCE_" + organ_name + ".png", bbox_inches='tight')


In [ ]:
# import skimage.transform
# import scipy.signal

# from matplotlib import pyplot as plt

# edge_ker = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])


# img512 = test_inputs[0,...,0].copy()

# img64 = skimage.transform.resize(img512, (64,64))
# kern64 = np.zeros_like(img64)
# kern64[8:11,8:11] = 1
# conv64 = scipy.signal.convolve2d(img64, edge_ker, mode='same', boundary='fill', fillvalue=0)
# up128 = skimage.transform.resize(conv64, (128,128))

# img32 = skimage.transform.resize(img512, (32,32))
# kern32 = np.zeros_like(img32)
# kern32[4:7,4:7] = 1
# conv32 = scipy.signal.convolve2d(img32, edge_ker, mode='same', boundary='fill', fillvalue=0)
# up64 = skimage.transform.resize(conv32, (64,64))

# img16 = skimage.transform.resize(img512, (16,16))
# kern16 = np.zeros_like(img16)
# kern16[2:5,2:5] = 1
# conv16 = scipy.signal.convolve2d(img16, edge_ker, mode='same', boundary='fill', fillvalue=0)
# up32 = skimage.transform.resize(conv16, (32,32))




# fig, axs = plt.subplots(nrows=3, ncols=3, squeeze=True, figsize=(12, 12))

# axs[0][0].imshow(img64)
# axs[0][0].imshow(kern64, alpha=0.4)
# axs[1][0].imshow(conv64)
# axs[2][0].imshow(up128)

# axs[0][1].imshow(img32)
# axs[0][1].imshow(kern32, alpha=0.4)
# axs[1][1].imshow(conv32)
# axs[2][1].imshow(up64)

# axs[0][2].imshow(img16)
# axs[0][2].imshow(kern16, alpha=0.4)
# axs[1][2].imshow(conv16)
# axs[2][2].imshow(up32)

# for b in axs:
#     for a in b:
#         a.set_xticklabels([])
#         a.set_yticklabels([])
#         a.set_xticks([])
#         a.set_yticks([])
    
# axs[0][0].set_title('Input 64x64', fontdict={'fontsize': 15, 'fontweight': 'medium'})
# axs[0][1].set_title('Input 32x32', fontdict={'fontsize': 15, 'fontweight': 'medium'})
# axs[0][2].set_title('Input 16x16', fontdict={'fontsize': 15, 'fontweight': 'medium'})

# axs[1][0].set_title('Feature extraction 64x64', fontdict={'fontsize': 15, 'fontweight': 'medium'})
# axs[1][1].set_title('Feature extraction 32x32', fontdict={'fontsize': 15, 'fontweight': 'medium'})
# axs[1][2].set_title('Feature extraction 16x16', fontdict={'fontsize': 15, 'fontweight': 'medium'})

# axs[2][0].set_title('Upsample 64x64 to 128x128', fontdict={'fontsize': 15, 'fontweight': 'medium'})
# axs[2][1].set_title('Upsample 32x32 to 64x64', fontdict={'fontsize': 15, 'fontweight': 'medium'})
# axs[2][2].set_title('Upsample 16x16 to 32x32', fontdict={'fontsize': 15, 'fontweight': 'medium'})

# plt.tight_layout()

# plt.savefig("encode_block" + ".svg", bbox_inches='tight')

In [ ]:

# from matplotlib import pyplot as plt

# for index in range(len(test_inputs)):
#     img = test_inputs[index:index+1]
#     truth = test_truths[index:index+1]
#     pred = model.predict(img)
#     pred = np.round(pred)

#     plt.figure(figsize=(5,5))
#     plt.imshow(img[0,...,0], cmap='gray')
#     plt.imshow(truth[0,...,0], alpha=0.25)
#     plt.imshow(2*pred[0,...,0], alpha=0.25)
#     plt.show()

In [ ]:
# for index in range(len(test_inputs)):
#     img = test_inputs[index:index+1]
#     truth = test_truths[index:index+1]
#     pred = model.predict(img)
#     pred = np.round(pred)
    
#     plt.figure(figsize=(5,5))
#     plt.imshow(img[0,...,0], cmap='gray')
#     plt.imshow(truth[0,...,1], alpha=0.25)
#     plt.imshow(2*pred[0,...,1], alpha=0.25)
#     plt.show()

In [ ]:
from matplotlib import pyplot as plt

p = []
t = []

for index in range(len(test_inputs)):
    img = test_inputs[index:index+1]
    truth = test_truths[index:index+1]
    pred = model.predict(img)
    pred = np.round(pred)
    
    
    fig, axs = plt.subplots(nrows=1, ncols=2, squeeze=True, figsize=(6, 12))

    axs[0].imshow(truth[0,...,2])
    axs[1].imshow(pred[0,...,2])
    plt.show()

In [ ]:
# from matplotlib import pyplot as plt

# random_indexes = np.random.randint(0, test_inputs.shape[0], 5)
# #random_indexes = [107, 120, 61, 95, 185]
# number = len(random_indexes)
# print(random_indexes)
# fig, axs = plt.subplots(nrows=number, ncols=4, sharex=True, sharey=True, squeeze=True, figsize=(12,15))
# for j, index in enumerate(random_indexes):
#     ax = axs[j]
    
#     for a in ax:
#         a.set_xticklabels([])
#         a.set_yticklabels([])
#         a.set_xticks([])
#         a.set_yticks([])
        
#     img = test_inputs[index:index+1]
#     truth = test_truths[index:index+1]
#     pred = model.predict(img)
#     pred = np.round(pred)
#     diff = truth - pred
    
#     ax[0].imshow(np.log(1+img[0,...,0]))
#     ax[1].imshow(truth[0,...,0])
#     ax[2].imshow(pred[0,...,0])
#     ax[3].imshow(diff[0,...,0])
# plt.tight_layout()

# axs[0][0].set_title('Input', fontdict={'fontsize': 15, 'fontweight': 'medium'})
# axs[0][1].set_title('Truth', fontdict={'fontsize': 15, 'fontweight': 'medium'})
# axs[0][2].set_title('Output', fontdict={'fontsize': 15, 'fontweight': 'medium'})
# axs[0][3].set_title('Difference', fontdict={'fontsize': 15, 'fontweight': 'medium'})

# plt.savefig("vacbag.svg", bbox_inches='tight')

In [ ]:
preds = []
for index in range(len(test_inputs)):
    img = test_inputs[index:index+1]
    pred = model.predict(img)
    preds.append(pred[0])
    
preds = np.array(preds)
preds = np.round(preds)


In [ ]:
print(preds.shape)
print(test_inputs.shape)
print(test_truths.shape)

index = 1
organ = 2

plt.figure()
plt.imshow(preds[index,...,organ])
plt.figure()
plt.imshow(test_truths[index,...,organ])

In [ ]:
spacing_mm = np.array([1.3671875, 1.3671875, 1])

rectum_mean = 2.862
rectum_std = 2.066

bladder_mean = 0.99
bladder_std = np.std(np.array([.23, .2, .1, .76, .21]))

# patient_tol_mm = 1
rectum_tol_mm = rectum_mean + 2 * rectum_std
bladder_tol_mm = bladder_mean + 2 * bladder_std
rectum_tol_mm, bladder_tol_mm

In [ ]:
rectum_tol_mm, bladder_tol_mm

In [ ]:
patient_truths = test_truths[...,0:1]
bladder_truths = test_truths[...,1:2]
rectum_truths = test_truths[...,2:3]

patient_preds = preds[...,0:1]
bladder_preds = preds[...,1:2]
rectum_preds = preds[...,2:3]

print(bladder_truths.shape, rectum_truths.shape)
print(bladder_preds.shape, rectum_preds.shape)

In [ ]:
from matplotlib import pyplot as plt

index = 1

plt.figure()
plt.imshow(rectum_truths[index,...,0])
plt.figure()
plt.imshow(rectum_preds[index,...,0])

In [ ]:
# num_bladders = 0
# for x in bladder_truths:
#     if np.max(x) > 0.5:
#         num_bladders += 1
# print(num_bladders)

# num_rectums = 0
# for x in rectum_truths:
#     if np.max(x) > 0.5:
#         num_rectums += 1
# print(num_rectums)

In [ ]:
print(loss.dsc(patient_truths, patient_preds))
print(loss.dice_metric(patient_truths, patient_preds))

In [ ]:
print(loss.dsc(bladder_truths, bladder_preds))
print(loss.dice_metric(bladder_truths, bladder_preds))

In [ ]:
print(loss.dsc(rectum_truths, rectum_preds))
print(loss.dice_metric(rectum_truths, rectum_preds))

In [ ]:
from metrics import *

def google_metrics(y_true, y_pred, spacing_mm, organ_tol_mm, organ_index=None):
    if organ_index == None:
        y_pred = np.array(y_pred, dtype=bool)
        y_true = np.array(y_true, dtype=bool)
    else:
        y_pred = np.array(y_pred[...,organ_index], dtype=bool)
        y_true = np.array(y_true[...,organ_index], dtype=bool)
    surface_dice_list = []
    msd_list = []
    dice_list = []

    for true, pred in zip(y_true, y_pred):

        
        if np.max(true) == 1 and np.max(pred) == 1:
            surface_distances = compute_surface_distances(pred, true, spacing_mm)  
            
            surface_dice = compute_surface_dice_at_tolerance(surface_distances, organ_tol_mm)
            surface_dice_list.append(surface_dice)
            
            msd = np.mean(compute_average_surface_distance(surface_distances))
            msd_list.append(msd)
            
            dice = compute_dice_coefficient(true, pred)
            dice_list.append(dice)
            
            
        elif np.max(true) == 1 and np.max(pred) == 0:
            #msd_list.append(9999)
            surface_dice_list.append(0.00)
            dice_list.append(0.00)
            
#         elif np.max(true) < 1 and np.max(pred) < 1:
#             #msd_list.append(0.00)
#             surface_dice_list.append(1.00)          
#             dice_list.append(1.00)
#             pass
        
        elif np.max(true) == 0 and np.max(pred) ==1:
            surface_dice_list.append(0.00)
            dice_list.append(0.00)
        else:
            print("error")
            
            
    return dice_list, surface_dice_list, msd_list

In [ ]:
stats_patient = google_metrics(patient_truths, patient_preds, spacing_mm, 1.3671875)
for metric in stats_patient:
    print(np.mean(metric), np.std(metric))
    
myNumber = np.min(stats_patient[1])
print(myNumber)
find = min(stats_patient[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats_patient[1]) if e == find])
print("-------")

myNumber = np.max(stats_patient[1])
print(myNumber)
find = min(stats_patient[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats_patient[1]) if e == find])
print("-------")

myNumber = np.mean(stats_patient[1])
print(myNumber)
find = min(stats_patient[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats_patient[1]) if e == find])
print("-------")

myNumber = np.mean(stats_patient[1]) + np.std(stats_patient[1])
print(myNumber)
find = min(stats_patient[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats_patient[1]) if e == find])
print("-------")

myNumber = np.mean(stats_patient[1]) - np.std(stats_patient[1])
print(myNumber)
find = min(stats_patient[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats_patient[1]) if e == find])
print("-------")

In [ ]:
stats_bladder = google_metrics(bladder_truths, bladder_preds, spacing_mm, bladder_tol_mm)
for metric in stats_bladder:
    print(np.mean(metric), np.std(metric))
    
stats = stats_bladder    
    
myNumber = np.min(stats[1])
print(myNumber)
find = min(stats[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats[1]) if e == find])
print("-------")
myNumber = np.max(stats[1])
#myNumber = 0.97759001
print(myNumber)
find = min(stats_patient[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats[1]) if e == find])
print([(i,e) for i, e in enumerate(stats[1])])
print("-------")

myNumber = np.mean(stats[1])
print(myNumber)
find = min(stats[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats[1]) if e == find])

print("-------")

myNumber = np.mean(stats[1]) + np.std(stats[1])
print(myNumber)
find = min(stats[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats[1]) if e == find])
print("-------")

myNumber = np.mean(stats[1]) - np.std(stats[1])
print(myNumber)
find = min(stats[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats[1]) if e == find])
print("-------")


print(np.sort(stats[1]))

In [ ]:
stats_rectum = google_metrics(rectum_truths, rectum_preds, spacing_mm, rectum_tol_mm)
for metric in stats_rectum:
    print(np.mean(metric), np.std(metric))
    
    
stats = stats_rectum   
    
myNumber = np.min(stats[1])
print(myNumber)
find = min(stats[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats[1]) if e == find])
print("-------")

myNumber = np.max(stats[1])
print(myNumber)
find = min(stats_patient[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats[1]) if e == find])
print("-------")

myNumber = np.mean(stats[1])
print(myNumber)
find = min(stats[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats[1]) if e == find])
print("-------")

myNumber = np.mean(stats[1]) + np.std(stats[1])
print(myNumber)
find = min(stats[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats[1]) if e == find])
print("-------")

myNumber = np.mean(stats[1]) - np.std(stats[1])
print(myNumber)
find = min(stats[1], key=lambda x:abs(x-myNumber))
print(find)
print([i for i, e in enumerate(stats[1]) if e == find])
print("-------")

In [ ]:
print(rectum_tol_mm, bladder_tol_mm)

In [ ]:
np.std(stats_rectum[1] + stats_bladder[1] + stats_patient[1])

In [ ]:
import medpy.metric.binary

In [ ]:
print(medpy.metric.binary.dc(patient_preds, patient_truths))
print(medpy.metric.binary.sensitivity(patient_preds, patient_truths))
print(medpy.metric.binary.specificity(patient_preds, patient_truths))

In [ ]:
print(medpy.metric.binary.dc(bladder_preds, bladder_truths))
print(medpy.metric.binary.sensitivity(bladder_preds, bladder_truths))
print(medpy.metric.binary.specificity(bladder_preds, bladder_truths))

In [ ]:
print(medpy.metric.binary.dc(rectum_preds, rectum_truths))
print(medpy.metric.binary.sensitivity(rectum_preds, rectum_truths))
print(medpy.metric.binary.specificity(rectum_preds, rectum_truths))